In [1]:
import pandas as pd
import mysql.connector

In [ ]:
# Load your dataset
providers = pd.read_csv(r"C:\Users\uniq\Downloads\providers_data.csv")  ##providers data
receivers =pd.read_csv(r"C:\Users\uniq\Downloads\receivers_data.csv") ##recivers data 
food_listings =pd.read_csv(r"C:\Users\uniq\Downloads\food_listings_data.csv") ##food listings data
claims=pd.read_csv(r"C:\Users\uniq\Downloads\claims_data.csv") ##claims data


In [ ]:
providers

In [4]:
## mysql connection setup

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Santoshkumar$1979",
)
cursor = conn.cursor()

In [5]:
cursor.execute("CREATE DATABASE IF NOT EXISTS food_wastage")
print("MYSQL database 'food_wastage' created successfully")

MYSQL database 'food_wastage' created successfully


In [6]:
cursor.execute("use food_wastage")

In [7]:
###schema of table
cursor.execute (""" 
   CREATE TABLE IF NOT EXISTS providers (
       Provider_ID INT PRIMARY KEY,
       Name VARCHAR(255),
       Type VARCHAR(100),
       Address TEXT,
       City VARCHAR(100),
       Contact VARCHAR(50)
    )
""")
conn.commit()

In [8]:
##insert data using iterrows
for index, row in providers.iterrows():
    cursor.execute("""
        INSERT INTO providers (Provider_ID , Name , Type, Address, City, Contact)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Name = VALUES(Name),
            Type = VALUES(Type),
            Address = VALUES(Address),
            City = VALUES(City),
            Contact = VALUES(Contact)
    """, tuple(row))
conn.commit()


In [9]:
receivers

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154
...,...,...,...,...,...
995,996,Matthew Curtis,Shelter,Lanechester,127-889-4442x1289
996,997,Amanda Cain,NGO,New Steven,+1-001-491-5601x5316
997,998,Theodore Briggs,Individual,South Sandra,930-609-9442x5031
998,999,Cheyenne Ramsey,NGO,Lake Jeffery,001-326-320-4816x15300


In [10]:
cursor.execute (""" 
   CREATE TABLE IF NOT EXISTS receivers (
       Receiver_ID  INT PRIMARY KEY,
       Name VARCHAR(255),
       Type VARCHAR(100),
       Address TEXT,
       City VARCHAR(100),
       Contact VARCHAR(50)
    )
""")
conn.commit()

In [11]:
##insert data using iterrows
for index, row in receivers.iterrows():
    cursor.execute("""
        INSERT INTO receivers (Receiver_ID , Name , Type, City, Contact)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Name = VALUES(Name),
            Type = VALUES(Type),
            City = VALUES(City),
            Contact = VALUES(Contact)
    """, tuple(row))
conn.commit()

In [12]:
food_listings

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,3/28/2025,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,3/16/2025,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,3/19/2025,279,Restaurant,Garciaport,Vegan,Dinner
...,...,...,...,...,...,...,...,...,...
995,996,Fish,15,3/30/2025,467,Catering Service,Phillipsfort,Vegan,Breakfast
996,997,Fish,22,3/18/2025,35,Grocery Store,Andersonmouth,Vegetarian,Breakfast
997,998,Fruits,6,3/22/2025,444,Restaurant,New Billy,Non-Vegetarian,Dinner
998,999,Pasta,15,3/30/2025,702,Supermarket,Lake Mistyton,Non-Vegetarian,Lunch


In [13]:
cursor.execute (""" 
   CREATE TABLE IF NOT EXISTS food_listings (
       Food_ID  INT PRIMARY KEY,
       Food_Name VARCHAR(100),
       Quantity FLOAT,
       Expiry_Date DATE,
       Provider_ID 	INT,
       Provider_Type VARCHAR(50),
       Location  VARCHAR(100),
       Food_Type VARCHAR(50),
       Meal_Type VARCHAR(50)
    )
""")
conn.commit()

In [14]:

# Convert Expiry_Date to datetime object
food_listings["Expiry_Date"] = pd.to_datetime(food_listings["Expiry_Date"], errors="coerce")

# Format it to MySQL's accepted date string format
food_listings["Expiry_Date"] = food_listings["Expiry_Date"].dt.strftime('%Y-%m-%d')


In [15]:
##insert data using iterrows
for index, row in food_listings.iterrows():
    cursor.execute("""
        INSERT INTO food_listings (
            Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID,
            Provider_Type, Location, Food_Type, Meal_Type
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Food_Name = VALUES(Food_Name),
            Quantity = VALUES(Quantity),
            Expiry_Date = VALUES(Expiry_Date),
            Provider_ID = VALUES(Provider_ID),
            Provider_Type = VALUES(Provider_Type),
            Location = VALUES(Location),
            Food_Type = VALUES(Food_Type),
            Meal_Type = VALUES(Meal_Type)
    """, tuple(row))
    conn.commit()


In [16]:
claims

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59
3,4,61,933,Cancelled,3/4/2025 9:08
4,5,345,229,Pending,3/14/2025 15:17
...,...,...,...,...,...
995,996,855,211,Completed,3/13/2025 19:40
996,997,980,746,Cancelled,3/17/2025 22:04
997,998,832,967,Cancelled,3/13/2025 18:00
998,999,917,90,Completed,3/1/2025 15:31


In [17]:
cursor.execute (""" 
   CREATE TABLE IF NOT EXISTS claims (
       Claim_ID  INT PRIMARY KEY,
       Food_ID INT,
       Receiver_ID INT,
       Status VARCHAR(100),
       Timestamp DATETIME
    )
""")
conn.commit()

In [18]:
# Convert Timestamp column to proper datetime object
claims["Timestamp"] = pd.to_datetime(claims["Timestamp"], errors='coerce')

# Format datetime to MySQL-compatible string format
claims["Timestamp"] = claims["Timestamp"].dt.strftime('%Y-%m-%d %H:%M:%S')


In [19]:
##insert data using iterrows
for index, row in claims.iterrows():
    cursor.execute("""
        INSERT INTO claims (Claim_ID , Food_ID , Receiver_ID, Status, Timestamp)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Claim_ID = VALUES(Claim_ID),
            Food_ID = VALUES(Food_ID),
            Receiver_ID = VALUES(Receiver_ID),
            Status = VALUES(Status),
            Timestamp = VALUES(Timestamp)
    """, tuple(row))
conn.commit()

In [3]:
import mysql.connector
import pandas as pd
from tabulate import tabulate
# Step 1: Connect to database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Santoshkumar$1979",
    database="food_wastage"
)

# Step 2: Create a cursor
cursor = conn.cursor()

In [19]:
###1How many food providers and receivers are there in each city?  
query = """
SELECT 
    city,
    SUM(provider_count) AS provider_count,
    SUM(receiver_count) AS receiver_count
FROM (
    SELECT city, COUNT(*) AS provider_count, 0 AS receiver_count
    FROM providers
    GROUP BY city

    UNION ALL

    SELECT city, 0 AS provider_count, COUNT(*) AS receiver_count
    FROM receivers
    GROUP BY city
) AS combined
GROUP BY city
ORDER BY provider_count DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

df = pd.DataFrame(results, columns=['city', 'provider_count', 'receiver_count'])
print(df)

                          city provider_count receiver_count
0                    New Carol              3              0
1     South Christopherborough              3              0
2                   New Amanda              2              0
3             North Kevinhaven              2              0
4                    Lake Kyle              2              0
...                        ...            ...            ...
1873               Andrewsport              0              1
1874               Lanechester              0              1
1875              South Sandra              0              1
1876              Lake Jeffery              0              1
1877             Shelbychester              0              1

[1878 rows x 3 columns]


In [17]:
###1...Which city has the highest number of food providers
query = """SELECT City,COUNT(*) AS Provider_Count
       FROM providers
       GROUP BY City 
       ORDER BY Provider_Count DESC 
       LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()

df=pd.DataFrame(result, columns=["City", "Provider_Count"])
df

,City,Provider_Count
0,New Carol,3


In [18]:
##2Which type of food provider (restaurant, grocery store, etc.) contributes the most food?
query = """
    SELECT provider_type, COUNT(*) AS Contribution_Count
    FROM food_listings
    GROUP BY provider_type
    ORDER BY Contribution_Count DESC
    LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Provider_Type", "Contribution_Count"])
df


,Provider_Type,Contribution_Count
0,Supermarket,267


In [25]:
##3.What is the contact information of food providers in a specific city?

city_name = "New Jessica"  # Change to any city

query = f"""
SELECT name, contact
FROM providers
WHERE city = '{city_name}';
"""

cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Name", "Contact"])
df



,Name,Contact
0,Gonzales-Cochran,+1-600-220-0480


In [27]:
##4.Which receivers have claimed the most food?
query = """
    SELECT r.Name, COUNT(*) AS Claim_Count
    FROM claims c
    JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
    GROUP BY r.Name
    ORDER BY Claim_Count DESC
    LIMIT 1;
"""

cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Receiver_Name", "Claim_Count"])
df

,Receiver_Name,Claim_Count
0,William Frederick,5


In [28]:
##5.What is the total quantity of food available from all providers?
query = """
    SELECT SUM(quantity) AS Total_Quantity
    FROM food_listings;
"""

cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Total_Quantity"])
df


,Total_Quantity
0,25794.0


In [30]:
##6.Which city has the highest number of food listings?
query = """
    SELECT Location AS City, COUNT(*) AS Listing_Count
    FROM food_listings
    GROUP BY Location
    ORDER BY Listing_Count DESC
    LIMIT 1;
"""

cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["City", "Listing_Count"])
df

,City,Listing_Count
0,New Carol,6


In [31]:
##7.What are the most commonly available food types?
query = """
    SELECT Food_Type, COUNT(*) AS Frequency
    FROM food_listings
    GROUP BY Food_Type
    ORDER BY Frequency DESC;
"""

cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Food_Type", "Frequency"])
df

,Food_Type,Frequency
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


In [32]:
##8. How many food claims have been made for each food item?
query = """
    SELECT Food_ID, COUNT(*) AS Claim_Count
    FROM claims
    GROUP BY Food_ID
    ORDER BY Claim_Count DESC;
"""

cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Food_ID", "Claim_Count"])
df

,Food_ID,Claim_Count
0,486,5
1,463,5
2,548,5
3,674,4
4,190,4
...,...,...
642,300,1
643,272,1
644,799,1
645,635,1


In [33]:
###9.Which provider has had the highest number of successful food claims?
query = """
    SELECT fl.Provider_ID, COUNT(*) AS Successful_Claims
    FROM claims c
    JOIN food_listings fl ON c.Food_ID = fl.Food_ID
    WHERE c.Status = 'Completed'
    GROUP BY fl.Provider_ID
    ORDER BY Successful_Claims DESC
    LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Provider_ID", "Successful_Claims"])
df

,Provider_ID,Successful_Claims
0,709,5


In [34]:
##10.What percentage of food claims are completed vs. pending vs. canceled?
query = """
    SELECT Status,
           COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS Percentage
    FROM claims
    GROUP BY Status;
"""
cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Status", "Percentage"])
df

,Status,Percentage
0,Pending,32.50000
1,Cancelled,33.60000
2,Completed,33.90000


In [5]:
##11.What is the average quantity of food claimed per receiver?
query = """
    SELECT r.Name,
           AVG(f.Quantity) AS Average_Quantity_Claimed
    FROM claims c
    JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
    JOIN food_listings f ON c.Food_ID = f.Food_ID
    GROUP BY r.Name;
"""
cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Receiver Name", "Average Quantity Claimed"])
df

,Receiver Name,Average Quantity Claimed
0,Abigail Crawford,25.666667
1,Regina Munoz,12.000000
2,Julie Lewis,37.000000
3,Jodi Lee,24.500000
4,Tina Watkins,14.000000
...,...,...
615,Rebecca Lucas,23.000000
616,Steve Kirby,26.000000
617,Judy Griffith,28.000000
618,Kenneth Barnes,2.000000


In [6]:
##12.Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?
query = """
    SELECT f.Meal_Type,
           COUNT(*) AS Total_Claims
    FROM claims c
    JOIN food_listings f ON c.Food_ID = f.Food_ID
    GROUP BY f.Meal_Type
    ORDER BY Total_Claims DESC;
"""
cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Meal Type", "Total Claims"])
df

,Meal Type,Total Claims
0,Breakfast,278
1,Lunch,250
2,Snacks,240
3,Dinner,232


In [7]:
##13.What is the total quantity of food donated by each provider?
query = """
    SELECT p.Name AS Provider_Name,
           SUM(f.Quantity) AS Total_Quantity_Donated
    FROM food_listings f
    JOIN providers p ON f.Provider_ID = p.Provider_ID
    GROUP BY p.Name
    ORDER BY Total_Quantity_Donated DESC;
"""
cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=["Provider Name", "Total Quantity Donated"])
df

,Provider Name,Total Quantity Donated
0,Miller Inc,217.0
1,Barry Group,179.0
2,"Evans, Wright and Mitchell",158.0
3,Smith Group,150.0
4,Campbell LLC,145.0
...,...,...
623,Reyes and Sons,1.0
624,Gonzales-Moore,1.0
625,Norris-Townsend,1.0
626,Mcgee PLC,1.0


In [61]:
!pip install streamlit

In [1]:
%%writefile app.py
import streamlit as st
import mysql.connector
import pandas as pd

# -------------------- DB CONNECTION --------------------
def get_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="Santoshkumar$1979",
        database="food_wastage"
    )
def get_data(query):
    try:
        conn = get_connection()
        df = pd.read_sql(query, conn)
        conn.close()
        return df
    except Exception as e:
        st.error(f"Query failed: {e}")
        return pd.DataFrame()

# -------------------- FETCH FULL TABLE --------------------
def fetch_table_data(table_name):
    query = f"SELECT * FROM {table_name}"
    return get_data(query)

# -------------------- STREAMLIT UI --------------------
st.set_page_config(layout="wide", page_title="Food Wastage Project")
st.title("🍲 Food_Wastage_Project")

# -------------------- SIDEBAR NAVIGATION --------------------
st.sidebar.title("📌 Navigation")
page = st.sidebar.radio("Go to", ["Project Introduction", "View Table", "SQL Query Visualizer", "Manage Data", "About the Developer"])

# -------------------- PROJECT INTRODUCTION --------------------
if page == "Project Introduction":
    st.subheader("📘 Project Introduction")
    st.write("""
       Food wastage is a growing concern in many communities, contributing to both environmental degradation and social inequality. 
    The **Local Food Wastage Management System** aims to address this issue by creating an efficient, technology-driven platform 
    that connects food donors with organizations or individuals in need, reducing surplus food waste at the local level.

    This system is designed to streamline the collection, management, and redistribution of excess food from restaurants, grocery stores, 
    households, and events. Through a user-friendly interface, donors can list surplus food, while NGOs, food banks, and volunteers can 
    claim and distribute it in real-time.

    The primary goal of the project is to minimize food wastage and promote sustainable community support.
        - 📦 **Food Listings**
        - 🧑‍🍳 **Providers**
        - 🧐 **Receivers**
        - 📁 **Claims made**
    """)

# -------------------- VIEW TABLE --------------------
elif page == "View Table":
    st.title("📊 View Table")

    def get_data(query):
        conn = get_connection()
        df = pd.read_sql(query, conn)
        conn.close()
        return df

    providers = get_data("SELECT * FROM providers")
    receivers = get_data("SELECT * FROM receivers")
    food_listings = get_data("SELECT * FROM food_listings")
    claims = get_data("SELECT * FROM claims")

    table_choice = st.selectbox("Select a dataset to view:", ["claims", "food_listings", "providers", "receivers"])
    st.write(f"📋 Showing data from `{table_choice}` table:")
    result = fetch_table_data(table_choice)
    if isinstance(result, str):
        st.error(result)
    else:
        st.dataframe(result, use_container_width=True)

# -------------------- SQL QUERY VISUALIZER --------------------
elif page == "SQL Query Visualizer":
    st.title("SQL Query Visualizer")
    
    queries = {
        "1. How many food providers and receivers are there in each city":
            "SELECT City, COUNT(*) AS Provider_Count FROM providers GROUP BY City ORDER BY Provider_Count DESC LIMIT 1",

        "2. Which type of food provider (restaurant, grocery store, etc.) contributes the most food":
            "SELECT provider_type, COUNT(*) AS Contribution_Count FROM food_listings GROUP BY provider_type ORDER BY Contribution_Count DESC LIMIT 1",

        "3. What is the contact information of food providers in a specific city":
            "SELECT name, contact FROM providers WHERE city = 'New Jessica'",

        "4. Which receivers have claimed the most food":
            "SELECT r.Name, COUNT(*) AS Claim_Count FROM claims c JOIN receivers r ON c.Receiver_ID = r.Receiver_ID GROUP BY r.Name ORDER BY Claim_Count DESC LIMIT 1",

        "5. What is the total quantity of food available from all providers":
            "SELECT SUM(quantity) AS Total_Quantity FROM food_listings",

        "6. Which city has the highest number of food listings":
            "SELECT Location AS City, COUNT(*) AS Listing_Count FROM food_listings GROUP BY Location ORDER BY Listing_Count DESC LIMIT 1",

        "7. What are the most commonly available food types":
            "SELECT Food_Type, COUNT(*) AS Frequency FROM food_listings GROUP BY Food_Type ORDER BY Frequency DESC",

        "8.How many food claims have been made for each food item":
           "SELECT Food_ID, COUNT(*) AS Claim_Count FROM claims GROUP BY Food_ID ORDER BY Claim_Count DESC",

        "9. Which provider has had the highest number of successful food claims":
            "SELECT fl.Provider_ID, COUNT(*) AS Successful_Claims FROM claims c JOIN food_listings fl ON c.Food_ID = fl.Food_ID WHERE c.Status = 'Completed' GROUP BY fl.Provider_ID ORDER BY Successful_Claims DESC LIMIT 1",

        "10. What percentage of food claims are completed vs. pending vs. canceled":
            "SELECT Status,COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS Percentage FROM claims GROUP BY Status",
        
        "11. What is the average quantity of food claimed per receiver":
             "SELECT r.Name,AVG(f.Quantity) AS Average_Quantity_Claimed FROM claims c JOIN receivers r ON c.Receiver_ID = r.Receiver_ID JOIN food_listings f ON c.Food_ID = f.Food_ID GROUP BY r.Name",

        "12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most":
             "SELECT f.Meal_Type,COUNT(*) AS Total_Claims FROM claims c JOIN food_listings f ON c.Food_ID = f.Food_ID GROUP BY f.Meal_Type ORDER BY Total_Claims DESC",

        "13. What is the total quantity of food donated by each provider":
             "SELECT p.Name AS Provider_Name,SUM(f.Quantity) AS Total_Quantity_Donated FROM food_listings f JOIN providers p ON f.Provider_ID = p.Provider_ID GROUP BY p.Name ORDER BY Total_Quantity_Donated DESC"
    }

    selected_query = st.selectbox("Choose a Query", list(queries.keys()))
    query_result = get_data(queries[selected_query])

    st.write("### Query Result:")
    st.dataframe(query_result)
#---------------------Developer SQL Queries--------------------

#---------------------MANAGE DATA -------------------------- 
elif page == "Manage Data":
    st.title("📦 Manage Data")

    entity = st.selectbox("Select Table", options=["providers", "receivers", "food_listings", "claims"])
    action = st.radio("Choose Action", ["Add", "Update", "Delete"])

    def run_query(query, values=None, fetch=False):
        try:
            conn = get_connection()
            cursor = conn.cursor(dictionary=True)
            if values:
                cursor.execute(query, values)
            else:
                cursor.execute(query)
            if fetch:
                result = cursor.fetchall()
            else:
                conn.commit()
                result = None
            cursor.close()
            conn.close()
            return result
        except Exception as e:
            st.error(f"Database error: {e}")
            return None
    # ✅ Move all action handlers under this `if` block
    if action == "Add":
        st.subheader(f"Add a new record to `{entity}`")
        columns = run_query(f"SHOW COLUMNS FROM {entity}", fetch=True)
        insert_data = {}
        for col in columns:
            if col['Field'].lower() == 'id':
                continue  # skip auto-increment id
            insert_data[col['Field']] = st.text_input(f"Enter {col['Field']}")

        if st.button("Add Record"):
            fields = ', '.join(insert_data.keys())
            placeholders = ', '.join(['%s'] * len(insert_data))
            values = list(insert_data.values())
            query = f"INSERT INTO {entity} ({fields}) VALUES ({placeholders})"
            run_query(query, values)
            st.success("Record added successfully!")

    # Update an existing record
    elif action == "Update":
        st.subheader(f"Update record in `{entity}`")
        data = run_query(f"SELECT * FROM {entity}", fetch=True)
        if not data:
            st.warning("No data found.")
        else:
            df = pd.DataFrame(data)
            st.dataframe(df)

            ids = [str(row['id']) for row in data]
            selected_id = st.selectbox("Select ID to update", ids)
            current = next(item for item in data if str(item['id']) == selected_id)

            updated_data = {}
            for col in df.columns:
                if col == 'id':
                    continue
                updated_data[col] = st.text_input(f"{col}", value=str(current[col]))

            if st.button("Update Record"):
                update_query = f"UPDATE {entity} SET " + ", ".join(f"{k}=%s" for k in updated_data.keys()) + " WHERE id=%s"
                run_query(update_query, list(updated_data.values()) + [selected_id])
                st.success("Record updated!")

    # Delete a record
    elif action == "Delete":
        st.subheader(f"Delete record from `{entity}`")
        data = run_query(f"SELECT * FROM {entity}", fetch=True)
        if not data:
            st.warning("No data found.")
        else:
            df = pd.DataFrame(data)
            st.dataframe(df)

            ids = [str(row['id']) for row in data]
            selected_id = st.selectbox("Select ID to delete", ids)

            if st.button("Delete Record"):
                run_query(f"DELETE FROM {entity} WHERE id = %s", (selected_id,))
                st.success("Record deleted.")

# -------------------- ABOUT DEVELOPER --------------------
elif page == "About the Developer":
    st.markdown("### 👤 About the Developer")
    st.markdown("""
    Hello! I'm the creator of the **Local Food Wastage Management System**.

    This project was built out of a passion for reducing food waste and building sustainable systems that connect those with surplus to those in need.

    **Name**: *BoorgulaSrithachandana*  
    **📧 Email**: *boorgulasrithachandana@gmail.com*    
    """)


Overwriting app.py


In [ ]:
!streamlit run app.py